### Data is from here https://e-service.cwb.gov.tw/HistoryDataQuery/index.jsp

In [1]:
import numpy as np
import pandas as pd
import requests
import os
from tqdm import tqdm

In [2]:
# 產生data List , data List為十年份
def date():
    month31=[1,3,5,7,8,10,12]
    month30=[4,6,9,11]
    year10=['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016' ,'2017']
    year_special = ['2008', '2012', '2016', '2020']
    nday31=range(1,32)
    nday30=range(1,31)
    nday29=range(1, 30)
    nday28=range(1,29)
    day10=['01','02','03','04','05','06','07','08','09']
    month12=day10+['10','11','12']
    nday31 = map(str,nday31[9:])
    nday30 = map(str,nday30[9:])
    nday29 = map(str,nday29[9:])
    nday28 = map(str,nday28[9:])
    day31 = day10 + list(nday31)
    day30 = day10 + list(nday30)
    day29 = day10 + list(nday29)
    day28 = day10 + list(nday28)
    output=[]
    s=""
    for year in year10:
        if year in year_special:
            for month,strmonth in zip(range(1,13),month12):
                if month in month31:
                    for day in day31:
                        s = year+'-'+strmonth+'-'+day
                        output.append(s)
                elif month in month30:
                    for day in day30:
                        s = year+'-'+strmonth+'-'+day
                        output.append(s)
                else:
                    for day in day29:
                        s = year+'-02-'+day
                        output.append(s)
        else:
            for month,strmonth in zip(range(1,13),month12):
                if month in month31:
                    for day in day31:
                        s = year+'-'+strmonth+'-'+day
                        output.append(s)
                elif month in month30:
                    for day in day30:
                        s = year+'-'+strmonth+'-'+day
                        output.append(s)
                else:
                    for day in day28:
                        s = year+'-02-'+day
                        output.append(s)
    return output

In [3]:
datepickers = date()
stations = ['C0A9A0', '466880', 'C0C480', '467571', 'C0E420', '467490', 'C0G650', 'C0K330', '467480', 'C0I110', '467410', '467440', 'C0R220', '467080', '466990', '467660', '467350', '467990','467110', '466940']
stnames = ['%25E5%25A4%25A7%25E7%259B%25B4', '%25E6%259D%25BF%25E6%25A9%258B', '%25E6%25A1%2583%25E5%259C%2592', '%25E6%2596%25B0%25E7%25AB%25B9', '%25E7%25AB%25B9%25E5%258D%2597', 
          '%25E8%2587%25BA%25E4%25B8%25AD', '%25E5%2593%25A1%25E6%259E%2597', '%25E8%2599%258E%25E5%25B0%25BE', '%25E5%2598%2589%25E7%25BE%25A9', '%25E7%25AB%25B9%25E5%25B1%25B1', 
          '%25E8%2587%25BA%25E5%258D%2597', '%25E9%25AB%2598%25E9%259B%2584', '%25E6%25BD%25AE%25E5%25B7%259E', '%25E5%25AE%259C%25E8%2598%25AD', '%25E8%258A%25B1%25E8%2593%25AE',
          '%25E8%2587%25BA%25E6%259D%25B1', '%25E6%25BE%258E%25E6%25B9%2596', '%25E9%25A6%25AC%25E7%25A5%2596', '%25E9%2587%2591%25E9%2596%2580', '%25E5%259F%25BA%25E9%259A%2586']

#台北市大直區,新北市板橋區,桃園市桃園,新竹縣新竹,苗栗縣竹南,台中市台中,彰化縣員林市,雲林縣虎尾鎮, 嘉義縣嘉義市,
#南投竹山,台南台南,高雄高雄,屏東潮州,宜蘭宜蘭,台東台東,澎湖澎湖,馬祖馬祖,金門金門,基隆基隆

In [4]:
def making_url():
    
    url_list = list()
    for station, stname in zip(stations, stnames):
        for datepicker in datepickers:
            url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station={}&stname={}&datepicker={}'.format(station, stname, datepicker)
            url_list.append(url)
        
    return url_list

In [5]:
url_list = making_url()

In [6]:
len(url_list)

73060

In [7]:
def find_data(url):
    df = pd.read_html(url)
    df = pd.DataFrame(df)
    
    place_time = df.iloc[0, 0]
    placename = place_time[0][0].split('_')[1]
    time = place_time[2][0].split(':')[1]
#     print (placename)
#     print (time)
    
    data = df.iloc[1, 0]
    data = pd.DataFrame(data.values[2:], columns=data.values[1])
    new_data = pd.DataFrame({'氣溫' : data['氣溫(℃)Temperature']})   #, '降水量' : data['降水量(mm)Precp']
    
    temp = []
    for i in range(len(new_data)):
        if new_data.loc[:, '氣溫'][i] != 'X':
            temp.append(float(new_data.loc[:, '氣溫'][i]))
        
    maximun = max(temp)
    minimun = min(temp)
    mean_ = np.mean(temp)
    
    final_data = pd.DataFrame({'地區':[placename], '時間':[time], 'max_temp':[maximun], 'min_temp':[minimun], 'average_temp':[mean_]})
    final_data = final_data.loc[:, ['時間', '地區', 'max_temp', 'min_temp', 'average_temp']]
    final_data = final_data.dropna()
    
    return final_data

In [8]:
# 3003~3005 17927~17935 27689~27691 35692都XX

### 檢查的部分

In [9]:
# !pip install lxml

    100% |████████████████████████████████| 5.7MB 85kB/s eta 0:00:01
You are using pip version 9.0.1, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
find_data(url_list[3630])

,時間,地區,max_temp,min_temp,average_temp
0,2017-12-09,大直,17.3,13.8,15.325


### 13hours...

In [11]:
all_data = pd.DataFrame({})
for url in tqdm(url_list[:2]):
    final_data = find_data(url)
    all_data = pd.concat([all_data, final_data], axis=0)

all_data.head()

100%|██████████| 2/2 [00:02<00:00,  1.49s/it]


,時間,地區,max_temp,min_temp,average_temp


In [12]:
# all_data.to_csv('all_data.csv', index=False)
# all_data.tail()